In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24017
2,Huelva,Confirmados PDIA 14 días,2129
3,Huelva,Tasa PDIA 14 días,"414,872264551708"
4,Huelva,Confirmados PDIA 7 días,609
5,Huelva,Total Confirmados,24248
6,Huelva,Curados,13422
7,Huelva,Fallecidos,255


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24017.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6397.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 745 personas en los últimos 7 días 

Un positivo PCR cada 263 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24017.0,609.0,2129.0,513170.0,118.674124,414.872265,26.0
Huelva-Costa,14625.0,400.0,1311.0,289548.0,138.146352,452.774669,17.0
Condado-Campiña,6641.0,161.0,633.0,156231.0,103.052531,405.169269,8.0
Huelva (capital),6397.0,193.0,547.0,143837.0,134.179662,380.291580,6.0
Isla Cristina,1964.0,48.0,164.0,21393.0,224.372458,766.605899,5.0
Rociana del Condado,298.0,11.0,41.0,7939.0,138.556493,516.437839,5.0
Cartaya,877.0,23.0,78.0,20083.0,114.524722,388.388189,3.0
Punta Umbría,638.0,22.0,54.0,15355.0,143.275806,351.676978,2.0
Sierra de Huelva-Andévalo Central,2323.0,36.0,110.0,67391.0,53.419596,163.226544,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,163.0,7.0,41.0,2885.0,242.634315,1421.143847,0.0
Alájar,16.0,1.0,7.0,759.0,131.752306,922.266140,NaN
Villalba del Alcor,437.0,1.0,26.0,3366.0,29.708853,772.430184,0.0
Isla Cristina,1964.0,48.0,164.0,21393.0,224.372458,766.605899,5.0
Lepe,1944.0,59.0,181.0,27880.0,211.621234,649.210904,1.0
Ayamonte,1169.0,26.0,130.0,21104.0,123.199393,615.996967,0.0
Villanueva de los Castillejos,143.0,5.0,17.0,2825.0,176.991150,601.769912,0.0
Bollullos Par del Condado,570.0,21.0,86.0,14387.0,145.965107,597.761868,2.0
Palma del Condado (La),960.0,7.0,61.0,10801.0,64.808814,564.762522,-1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villalba del Alcor,437.0,1.0,26.0,3366.0,29.708853,772.430184,0.0,0.038462
Trigueros,221.0,1.0,20.0,7862.0,12.719410,254.388196,-1.0,0.050000
San Bartolomé de la Torre,189.0,1.0,13.0,3761.0,26.588673,345.652752,0.0,0.076923
Niebla,158.0,1.0,9.0,4116.0,24.295432,218.658892,0.0,0.111111
Palma del Condado (La),960.0,7.0,61.0,10801.0,64.808814,564.762522,-1.0,0.114754
San Juan del Puerto,353.0,4.0,32.0,9411.0,42.503453,340.027627,2.0,0.125000
Alájar,16.0,1.0,7.0,759.0,131.752306,922.266140,0.0,0.142857
Almonaster la Real,55.0,1.0,6.0,1788.0,55.928412,335.570470,-1.0,0.166667
Beas,159.0,4.0,24.0,4341.0,92.144667,552.868003,1.0,0.166667
